In [1]:
import pickle
import numpy as np
from scipy.linalg import norm
import pandas as pd
import plotly.graph_objs as go


In [2]:
# Author Andy Webber andygwebber@yahoo.com

# Parameters Author believes users will be most likely to play with
relative_threshold = 0.020 # The relative threshold of for drawing curves in Sankey diagrams
                 # 0.02 means all values of 0.2 multiplied by the largest transfer will be drawn
                 # A lower threshold will lead to a buisier plot
black_white = False # Black and White versions option
labeled = True  # Author likes unlabeled versions as well

# Parameters of the paper that should not be changed
N1S = 48 # Number of streamwise wavenumbers in tracking subspace
N3S = 59 # Number of spanwise wavenumbers in tracking subspace
X1L = 12.56637 # Streamwise length of the channel
X3L = 6.28319 # Spanwise widgth of the channel
pi = 3.14159

In [3]:
def node_swap(transfer_matrix, nodes, sink, source):
    """ This function tests if swap will help reduce the negatives
        and if so swaps the sink and the source nodes """

    triangle_sum = 0.0
    for isource in range(source, sink):
        triangle_sum += transfer_matrix[sink][isource]
    for isink in range(source + 1, sink):
        triangle_sum += transfer_matrix[isink][source]
    if triangle_sum > 0:
        """ This is the case where switching did not help so just return
            the transfer matrix as is and value at sink, source are added to acceptable_negatives"""
        return transfer_matrix[sink][source]
    else:
#        print(triangle_sum, sink, source, nodes[sink], nodes[source])
        """ This is the case where switching does help so do the switch """
        temp = nodes[sink]
        nodes[sink] = nodes[source]
        nodes[source] = temp
        for col in range(source):
            temp = transfer_matrix[sink][col]
            transfer_matrix[sink][col] = transfer_matrix[source][col]
            transfer_matrix[source][col] = temp
        for row in range(sink + 1, len(nodes)):
            temp = transfer_matrix[row][sink]
            transfer_matrix[row][sink] = transfer_matrix[row][source]
            transfer_matrix[row][source] = temp
        """ finally flip the triangle """
        for i in range(1, sink - source):
            temp = transfer_matrix[source + i][source]
            transfer_matrix[source + i][source] = -1.0 * transfer_matrix[sink][source + i]
            transfer_matrix[sink][source + i] = -1.0 * temp
        transfer_matrix[sink][source] = -1.0 * transfer_matrix[sink][source]

        return 1.0

def optimal_order(transfer_matrix, nodes):
    """ This function takes in a transfer matrix along with nodes and
        reorders the matrix by switching the order of the nodes by swapping rows and columns to minimize
        the sum of negative values """

    max_iterations = 100
    acceptable_negatives = []
    minimium = -1
    iterations = 0
    while minimium < 0.0:
        minimium = transfer_matrix.max()
        for row in range(1,len(nodes)): # Start indexing at 1. The mother mode will always be node 0
            for col in range(1,row): # Again do not consider moving the mother mode in col 1
                if (transfer_matrix[row][col] < minimium) and (
                    transfer_matrix[row][col] not in acceptable_negatives
                ):
                    minimium = transfer_matrix[row][col]
                    sink = row
                    source = col

        value = node_swap(transfer_matrix, nodes, sink, source)
        if value < 0:
            acceptable_negatives.append(value)

        iterations += 1
        if iterations > max_iterations:
            break
    print("took " + str(iterations) + " iterations")
    return

In [4]:
def setup_nodes(df_nodes):
    """ This function sets up the nodes with position and color """
    node_x = []
    node_y = []
    xmin = 0.15
    xmax = 0.85
    ymin = 0.05
    ymax = 0.95
    ''' The x and y values of the mother mode'''
    node_x.append(xmin)
    node_y.append(0.5)

    delx = (xmax - xmin)/float(len(df_nodes)-1)
    num_y_vals = int((len(df_nodes) - 1)/2)
    dely = (ymax - ymin)/float(num_y_vals)
    for node in range(1,len(df_nodes)):
        offset = 0.5 * (ymax - ymin) * (node-1)/float(len(df_nodes)-2)
        node_x.append(xmin + float(node) * delx)
        sign = pow(-1, node)
        if sign > 0:
            node_y.append(ymax - offset)
        else:
            node_y.append(ymin + offset)

    node_color = []
    for i in range(len(df_nodes)):
        alpha = 1.0 - 0.5 * float(i)/float(len(df_nodes))
        color = 'rgba(255,0,0,' + str(alpha) + ')'
        if (black_white):
            color = 'rgba(100,100,100,.7)'
        node_color.append(color)

    node_label = []
    for i in range(len(df_nodes)):
        if (labeled):
            node_label.append(df_nodes.loc[i].at["Nodes"])
        else:
            node_label.append(' ')

    return(node_x, node_y, node_color, node_label)
    

In [5]:
def set_link_color(df_transfer):
    """ Sets the colors of the links """
    link_color = []
    for i in range(len(df_transfer)):
        color = 'rgba(0,100,0,0.2)'
        if (black_white):
            color = 'rgba(100,100,100,.7)'
        link_color.append(color)
        
    return(link_color)

In [6]:
def generate_figure(node_x, node_y, node_color, node_label, link_color, df_nodes, df_transfer):
    """ This function forms the final figure that is then saved to disk"""
    fig = go.Figure(data=[go.Sankey(
        # Define nodes
        node = dict(
          # These are all simple lists of length number of nodes or single value
          x = node_x,
          y = node_y,
          color =  node_color,
          thickness = 20 # Thickness must be a scalar. One thickness for all nodes
        ),

        # Add links
        link = dict(
        # These are all series with an index and value or single value
          source =  df_transfer.Source,
          target =  df_transfer.Target,
          value =  df_transfer.Value,
          color = link_color
    ))])

    # Label position and text is set
    annotations = []
    for i in range(1,len(df_nodes)):
        if (i%2 == 1):
            this_dict = dict(
                x=node_x[i],
                y=1.05-node_y[i],
                text=node_label[i],
                showarrow=False,
                xanchor = 'left',
                yanchor = 'middle'
            )
        else:
            this_dict = dict(
                x=node_x[i],
                y=.96-node_y[i],
                text=node_label[i],
                showarrow=False,
                xanchor = 'left',
                yanchor = 'middle'
            )
        annotations.append(this_dict)
    this_dict = dict(
            x=-.07,
            y=0.5,
            text=node_label[0],
            showarrow=False,
            xanchor = 'left',
            yanchor = 'middle'
    )
    annotations.append(this_dict)

    fig.update_layout(

        annotations= annotations
    )
# Again, do not judge by this figure, look at jpg and pdf output files

    return fig
                   

# User can uncomment this but know the author has found problems with diagrams produced in the notebooks.
# One should look at output files and ignore these images
#fig.show()

In [7]:
spanwise_transfer = pickle.load(open("spanwise_transfer.bin", "rb"))
spanwise_antisymmetry = norm(spanwise_transfer + spanwise_transfer.transpose())/norm (spanwise_transfer)
print("Normalized antisymmetry of spanwise transfer matrix is " + str(spanwise_antisymmetry))

spanwise_nodes = []
spanwise_nodes.append("mother")
for n in range(N3S):
    wavenumber = 2.0 * n * pi / X3L
    wavenumber_str = "k3 = " + str(round(wavenumber, 1))
    spanwise_nodes.append(wavenumber_str)

spanwise_transfer_ll = np.tril(spanwise_transfer) # Get a version of spanwise_transfer with only lower left elements
optimal_order(spanwise_transfer_ll, spanwise_nodes) # Swap rows and columns of the matrix and corresponding nodes to minimize the sum of 
                                                    # negative elements in lower left

maximium = max(spanwise_transfer_ll.max(), -1.0 * spanwise_transfer_ll.min())
threshold = relative_threshold * maximium

""" Compose arrays of Sources, Sinks, and Transfers for all transfers above the threshold """
Sources = []
Sinks = []
Transfers = []
largest_node = -1
for source in range(0, len(spanwise_nodes)):
    for sink in range(source + 1, len(spanwise_nodes)):
        value = spanwise_transfer_ll[sink, source]
        # All transfer values must be greater than zero. Optimal_order and eliminated almost all
        # negative values but where negative values remain, swap source, and sink and use absolute value.
        if abs(value) > threshold:
            largest_node = max(largest_node, source)
            largest_node = max(largest_node, sink)
            if value > 0:
                Sources.append(source)
                Sinks.append(sink)
                Transfers.append(value)
            else:
                Sources.append(sink)
                Sinks.append(source)
                Transfers.append(-1.0 * value)

df_transfer_spanwise = pd.DataFrame({"Source": Sources, "Target": Sinks, "Value": Transfers})

spanwise_nodes_used = spanwise_nodes[0 : largest_node + 1]
df_nodes_spanwise = pd.DataFrame(spanwise_nodes_used, columns=['Nodes'])

Normalized antisymmetry of spanwise transfer matrix is 1.0126447741356848e-06
took 101 iterations


In [8]:
spanwise_node_x, spanwise_node_y, node_color_spanwise, node_label_spanwise = setup_nodes(df_nodes_spanwise)
link_color_spanwise = set_link_color(df_transfer_spanwise)

In [9]:
fig_spanwise = generate_figure(spanwise_node_x, spanwise_node_y, node_color_spanwise, node_label_spanwise, link_color_spanwise, df_nodes_spanwise, df_transfer_spanwise)
# User can uncomment this but know the author has found problems with diagrams produced in the notebooks.
# One should look at jpg and pdf output files and ignore these images
#fig.show()

In [10]:
figure_file = 'spanwise_cascade.jpg'
fig_spanwise.write_image(figure_file, width=1400, height=775)
figure_file = 'spanwise_cascade.pdf'
fig_spanwise.write_image(figure_file, width=1200, height=775)

In [11]:
streamwise_transfer = pickle.load(open("streamwise_transfer.bin", "rb"))
streamwise_antisymmetry = norm(streamwise_transfer + streamwise_transfer.transpose())/norm (streamwise_transfer)
print("Normalized antisymmetry of streamwise transfer matrix is " + str(streamwise_antisymmetry))

streamwise_nodes = []
streamwise_nodes.append("mother")
for m in range(N1S):
    wavenumber = 2.0 * m * pi / X1L
    wavenumber_str = "k1 = " + str(round(wavenumber, 1))
    streamwise_nodes.append(wavenumber_str)

streamwise_transfer_ll = np.tril(streamwise_transfer) # Get a version of spanwise_transfer with only lower left elements
optimal_order(streamwise_transfer_ll, streamwise_nodes) # Swap rows and columns of the matrix and corresponding nodes to minimize the sum of 
                                                    # negative elements in lower left

maximium = max(streamwise_transfer_ll.max(), -1.0 * streamwise_transfer_ll.min())
threshold = relative_threshold * maximium

""" Compose arrays of Sources, Sinks, and Transfers for all transfers above the threshold """
Sources = []
Sinks = []
Transfers = []
largest_node = -1
for source in range(0, len(streamwise_nodes)):
    for sink in range(source + 1, len(streamwise_nodes)):
        value = streamwise_transfer_ll[sink, source]
        # All transfer values must be greater than zero. Optimal_order and eliminated almost all
        # negative values but where negative values remain, swap source, and sink and use absolute value.
        if abs(value) > threshold:
            largest_node = max(largest_node, source)
            largest_node = max(largest_node, sink)
            if value > 0:
                Sources.append(source)
                Sinks.append(sink)
                Transfers.append(value)
            else:
                Sources.append(sink)
                Sinks.append(source)
                Transfers.append(-1.0 * value)

df_transfer_streamwise = pd.DataFrame({"Source": Sources, "Target": Sinks, "Value": Transfers})

streamwise_nodes_used = streamwise_nodes[0 : largest_node + 1]
df_nodes_streamwise = pd.DataFrame(streamwise_nodes_used, columns=['Nodes'])

Normalized antisymmetry of streamwise transfer matrix is 1.3845920675812848e-06
took 1 iterations


In [12]:
streamwise_node_x, streamwise_node_y, node_color_streamwise, node_label_streamwise = setup_nodes(df_nodes_streamwise)
link_color_streamwise = set_link_color(df_transfer_streamwise)

In [13]:
fig_streamwise = generate_figure(streamwise_node_x, streamwise_node_y, node_color_streamwise, node_label_streamwise, link_color_streamwise, df_nodes_streamwise, df_transfer_streamwise)
# User can uncomment this but know the author has found problems with diagrams produced in the notebooks.
# One should look at jpg and pdf output files and ignore these images
#fig.show()

In [14]:
figure_file = 'streamwise_cascade.jpg'
fig_streamwise.write_image(figure_file, width=1400, height=775)
figure_file = 'streamwise_cascade.pdf'
fig_streamwise.write_image(figure_file, width=1200, height=775)